In [ ]:
import numpy as np
import tensorflow as tf

# Function to prepare the data for the entire dataset
def prepare_data_entire_dataset(texts, tokenizer):
    # Tokenize the entire dataset at once
    tokens = tokenizer(
        texts,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    # Keep default float32 for performance
    return {
        'input_ids': tokens['input_ids'],  # Keep float32 (default)
        'attention_mask': tokens['attention_mask']
    }

# List of classes (decoding from one-hot encoding)
classes = ohe.categories_[0].tolist()

# Function to make predictions on processed batch data
def make_predictions_batch(model, processed_data, batch_size, classes=classes):
    num_samples = processed_data['input_ids'].shape[0]
    predicted = []
    
    # Loop through the dataset in batches
    for i in range(0, num_samples, batch_size):
        # Slice the batch from the processed data
        batch_input_ids = processed_data['input_ids'][i:i + batch_size]
        batch_attention_mask = processed_data['attention_mask'][i:i + batch_size]
        
        # Prepare the batch input
        batch_processed_data = {
            'input_ids': batch_input_ids,
            'attention_mask': batch_attention_mask
        }
        
        # Predict on the current batch
        probs = model.predict_on_batch(batch_processed_data)
        
        # Append the predicted classes for the batch
        predicted.extend([classes[np.argmax(prob)] for prob in probs])
    
    return predicted

# List of texts to make predictions on
test_excert = test_data['cleaned_data'].to_list()

# Preprocess the entire dataset in one go
processed_data = prepare_data_entire_dataset(test_excert, tokenizer)

# Define the batch size for predictions
batch_size = 64  # Adjust based on system specs

# Make predictions in batches on the preprocessed dataset
predicted = make_predictions_batch(NewsClassificationBERTModel, processed_data, batch_size)

# Add predictions to the test_data DataFrame
test_data['predicted'] = predicted
